In [ ]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
from fiber.utils import Timer
import pickle

In [ ]:
#Load dataframe HT cohort
Cases_HT_beforeOnset2013= pd.read_pickle("./Cases_HT_beforeOnset2013.pkl")
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"onset_year": "HT_onset_year"})
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"age_in_days": "age_in_days_HT_onset"})

In [ ]:
Cases_HT_beforeOnset2013.filter(['age_in_days_HT_onset','onset_year', 'HT', 'age_in_days_delta', 'date_of_birth_actual' ])

In [ ]:
#Cases with disease development cohort extraction

#Load dataframe HT cohort
Cases_HT_beforeOnset2013= pd.read_pickle("./Cases_HT_beforeOnset2013.pkl")
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"onset_year": "HT_onset_year"})
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"age_in_days": "age_in_days_HT_onset"})

def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

cohort = df_to_cohort(Cases_HT_beforeOnset2013)

#heart disease
condition = (Diagnosis('I20.%', 'ICD-10')|
             Diagnosis('I21.%', 'ICD-10')|
             Diagnosis('I22.%', 'ICD-10')|
             Diagnosis('I23.%', 'ICD-10')|
             Diagnosis('I24.%', 'ICD-10')|
             Diagnosis('I25.%', 'ICD-10')|
             Diagnosis('I26.%', 'ICD-10')|
             Diagnosis('I27.%', 'ICD-10')|
             Diagnosis('I28.%', 'ICD-10')|
             Diagnosis('I30.%', 'ICD-10')|
             Diagnosis('I31.%', 'ICD-10')|
             Diagnosis('I32.%', 'ICD-10')|
             Diagnosis('I33.%', 'ICD-10')|
             Diagnosis('I34.%', 'ICD-10')|
             Diagnosis('I35.%', 'ICD-10')|
             Diagnosis('I36.%', 'ICD-10')|
             Diagnosis('I37.%', 'ICD-10')|
             Diagnosis('I38.%', 'ICD-10')|
             Diagnosis('I39.%', 'ICD-10')|
             Diagnosis('I40.%', 'ICD-10')|
             Diagnosis('I41.%', 'ICD-10')|
             Diagnosis('I42.%', 'ICD-10')|
             Diagnosis('I43.%', 'ICD-10')|
             Diagnosis('I44.%', 'ICD-10')|
             Diagnosis('I45.%', 'ICD-10')|
             Diagnosis('I46.%', 'ICD-10')|
             Diagnosis('I47.%', 'ICD-10')|
             Diagnosis('I48.%', 'ICD-10')|
             Diagnosis('I49.%', 'ICD-10')|
             Diagnosis('I50.%', 'ICD-10')|
             Diagnosis('I51.%', 'ICD-10')|
             Diagnosis('I52.%', 'ICD-10')|
             Diagnosis('410.%', 'ICD-9')|
             Diagnosis('411.%', 'ICD-9')|
             Diagnosis('412.%', 'ICD-9')|
             Diagnosis('413.%', 'ICD-9')|
             Diagnosis('414.%', 'ICD-9')|
             Diagnosis('415.%', 'ICD-9')|
             Diagnosis('416.%', 'ICD-9')|
             Diagnosis('417.%', 'ICD-9')|
             Diagnosis('420.%', 'ICD-9')|
             Diagnosis('421.%', 'ICD-9')|
             Diagnosis('422.%', 'ICD-9')|
             Diagnosis('423.%', 'ICD-9')|
             Diagnosis('424.%', 'ICD-9')|
             Diagnosis('425.%', 'ICD-9')|
             Diagnosis('426.%', 'ICD-9')|
             Diagnosis('427.%', 'ICD-9')|
             Diagnosis('428.%', 'ICD-9')|
             Diagnosis('I11.%', 'ICD-10')| #hypertensive heart disease
             Diagnosis('402.%', 'ICD-9')|  #hypertensive heart disease
             Diagnosis('429.%', 'ICD-9'))
  
Cases_HT_before2013_disease = cohort.get(condition)

Cases_HT_before2013_disease.set_index('medical_record_number', inplace = True)

### create new dataframe with only disease MRN and date_of_birth
Cases_HT_beforeOnset2013_DOB = Cases_HT_beforeOnset2013[['date_of_birth_actual','HT_onset_year', 'age_in_days_HT_onset']]
Cases_HT_before2013_disease_merged_DOB = Cases_HT_before2013_disease.merge(Cases_HT_beforeOnset2013_DOB, left_index=True, right_index=True, how="inner")

#add onset year
def add_onset_year(df_main):
    df_main['age_in_days_delta'] = pd.to_timedelta(df_main['age_in_days'],'d')
    df_main['onset_year'] = (df_main['date_of_birth_actual'] + df_main['age_in_days_delta']).dt.year
    return df_main

Cases_HT_before2013_disease_onset = add_onset_year(Cases_HT_before2013_disease_merged_DOB)
Cases_HT_before2013_disease_onset_unique = Cases_HT_before2013_disease_onset.groupby("medical_record_number").count()


In [ ]:
Cases_HT_before2013_disease_onset

In [ ]:
Cases_HT_before2013_disease_onset_unique

In [ ]:
#save as pkl files
Cases_HT_before2013_disease_onset.to_pickle("./Cases_HT_heart_disease_all.pkl")
Cases_HT_before2013_disease_onset_unique.to_pickle("./Cases_HT_heart_disease_all_unique.pkl")

In [ ]:
# Control Creation

In [ ]:
#Choose file to pick from
Cases_HT_beforeOnset2013= pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Cases_HT_beforeOnset2013.pkl")
Cases_complication = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/All/Cases_HT_heart_disease_all_unique.pkl")

#Create file with only MRNs for the cases
Cases_complication_OnlyMRNs = Cases_complication.drop(['age_in_days', 'context_name', 'context_diagnosis_code','date_of_birth_actual','age_in_days_delta', 'onset_year', 'HT_onset_year','age_in_days_HT_onset'], axis = 1) 

#Merge All Cases with only complication_MRNS
Controls_without_complication = Cases_HT_beforeOnset2013.merge(Cases_complication_OnlyMRNs, on ='medical_record_number', how='outer', indicator=True)

#Create new dataframe where MRNs that appear in both dataframes before are ignored
Controls = Controls_without_complication[Controls_without_complication._merge != 'both']


In [ ]:
Controls

In [ ]:
#name file correctly and save cohort
Controls.to_pickle("./Controls_No_Heart_Diseases_Ever.pkl")